In [22]:
import os
# Must be set before SparkContext / SparkSession is created!
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"


In [23]:
import requests
from datetime import datetime
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
import pandas as pd
# from delta.tables import DeltaTable

In [24]:
spark = SparkSession.builder \
    .appName("CM") \
    .getOrCreate()

In [ ]:
import requests
import base64
def get_cm_access_token():
    # Credentials
    password = ""
    username = "cmapi"
    # Token URL
    token_url = "https://rockpoolrac.residential.icarehealth.com.au/Tetra.Web/OAuth/Token"
    # Encode Basic Auth header
    credentials = f"{username}:{password}"
    encoded_credentials = base64.b64encode(credentials.encode("utf-8")).decode("utf-8")
    # Headers and form data
    headers = {
        "Authorization": f"Basic {encoded_credentials}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials"
    }
    # Token request
    response = requests.post(token_url, headers=headers, data=data)
    if response.status_code == 200:
        token = response.json().get("access_token")
        return token
    else:
        raise Exception(f"Failed to retrieve token: {response.status_code} - {response.text}")
# Test token retrieval
cm_access_token = get_cm_access_token()
if cm_access_token:
    print("CM token successfully retrieved")
else:
    print("Token retrieval failed")

CM token successfully retrieved


# Fetch API

In [26]:
BASE_URL = "https://rockpoolrac.residential.icarehealth.com.au/Tetra.Web"

In [27]:
headers = {
    "Authorization": f"Bearer {cm_access_token}",
    "Accept": "application/json",
}

url = f"{BASE_URL}/api/ExternalAssessmentForm/GetAssessmentFormResponses?formId=152890"
resp = requests.get(url, headers=headers)
resp.raise_for_status()
json_text = resp.text

In [28]:
record = resp.json()
if isinstance(record, dict) and "data" in record:
    records = record["data"]


# # Quick sanity check:
# for i, rec in enumerate(records[:5]):
#     print(i, type(rec))

Load to DF

In [29]:
# 3. Let Spark infer the schema from your JSON text
#    • We create a one‑row DF with a F.column "json", then ask for its schema.
schema = (
    spark
      .range(1)                                  # dummy DF with one row
      .select(F.schema_of_json(F.lit(json_text)).alias("inferred"))
      .collect()[0]["inferred"]                 # pull out the StructType
)


# 4. Create a DataFrame holding your JSON string
raw = spark.createDataFrame([(json_text,)], ["json"])

# 5. Parse it using the inferred schema
df = raw.select(F.from_json(F.col("json"), schema).alias("data")).select("data.*")


In [30]:
flattened = (
    df
    # explode the top-level `items` array
    .select(
        F.col("formId"),
        F.explode(F.col("items")).alias("item")
    )
    # explode each item's `responses` array
    .select(
        "formId",
        F.col("item.templateQuestion.text").alias("question_text"),
        F.explode(F.col("item.responses")).alias("resp")
    )
    # pull the response fields up
    .select(
        "formId",
        "question_text",
        F.col("resp.stringValue").alias("string_value"),
        F.col("resp.numericValue").alias("numeric_value"),
        F.col("resp.booleanValue").alias("boolean_value")
    )
)


In [31]:
# Show your fully‑flattened table
flattened.printSchema()
flattened.show(truncate=False)

root
 |-- formId: long (nullable = true)
 |-- question_text: string (nullable = true)
 |-- string_value: string (nullable = true)
 |-- numeric_value: double (nullable = true)
 |-- boolean_value: boolean (nullable = true)



Py4JJavaError: An error occurred while calling o299.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 26) (HL-WIN-018.modem executor driver): java.io.IOException: Cannot run program "/usr/bin/python3": CreateProcess error=2, The system cannot find the file specified
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1143)
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1073)
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:218)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:143)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:158)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:178)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:261)
	at org.apache.spark.api.python.PythonRDD.compute(Py

In [ ]:
pdf.head()

,resp_questionId,resp_responseId,resp_dateValue,resp_stringValue,resp_numericValue,resp_booleanValue,resp_score,resp_recordInformation.id,resp_recordInformation.userId,resp_recordInformation.userNameFirst,resp_recordInformation.userNameLast,resp_recordInformation.userQualification,resp_recordInformation.agencyStaffName,resp_recordInformation.agencyStaffDesignation,resp_recordInformation.dateCreated,meta_formId,meta_formVersion,meta_status,meta_dateCompleted,meta_items.templateQuestion.text
0,123,NaN,None,None,NaN,True,None,4121620,2821,Shuvekshya,Phuyal,Registered Nurse,None,None,2025-07-15T01:34:18.11Z,152890,1,1,2025-07-15T01:34:18Z,PART 1 - FALLS RISK STATUS
1,124,30965.0,None,None,NaN,None,8,4121621,2821,Shuvekshya,Phuyal,Registered Nurse,None,None,2025-07-15T01:34:18.11Z,152890,1,1,2025-07-15T01:34:18Z,Risk Factor: recent falls
2,125,30968.0,None,None,NaN,None,3,4121622,2821,Shuvekshya,Phuyal,Registered Nurse,None,None,2025-07-15T01:34:18.11Z,152890,1,1,2025-07-15T01:34:18Z,Risk factor: medications
3,126,30972.0,None,None,NaN,None,3,4121623,2821,Shuvekshya,Phuyal,Registered Nurse,None,None,2025-07-15T01:34:18.11Z,152890,1,1,2025-07-15T01:34:18Z,Risk factor: psychological
4,127,30976.0,None,None,NaN,None,3,4121624,2821,Shuvekshya,Phuyal,Registered Nurse,None,None,2025-07-15T01:34:18.11Z,152890,1,1,2025-07-15T01:34:18Z,Risk factor: cognitive status
